In [1]:
#!jupyter nbconvert --to=python TS_profile_python3.ipynb

In [ ]:
class TS_profile:   
    '''
    class Nut_depth(runname,resultpath,savepath,meshpath,ncfileWOA,first_year,last_year,
                 savefig=False,regional=True)
                 
    if regional = True, profiles will plotted for each main basins + Global Ocean. 
    Otherwise, just the Global Ocean.
    '''
    def __init__(self,resultpath,savepath,mesh,ncfileTEMP,ncfileSAL,
                 first_year,last_year,savefig=False, regional=True, maxdepth = 5,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncfileTEMP = ncfileTEMP
        self.ncfileSAL = ncfileSAL
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.regional = regional
        self.maxdepth = maxdepth
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_WOA import WOAdata  
        
        years = np.arange(first_year, last_year+1,1)
        meshdiag = pf.get_meshdiag(mesh)
        runid      =  self.runname
        
        labelfesom = 'FESOM'
        labelwoa = 'WOA'
        unittemp = 'T [$^{\circ}$C]'
        unitsal = 'Salinity'
        
        # load data -------------------------------------------------------------------------------------
        TEMPfesom = pf.get_data(resultpath, "temp", years, mesh,
                               how="mean", compute=True, runid=runid, silent=True)

        SALfesom = pf.get_data(resultpath, "salt", years, mesh,
                               how="mean", compute=True, runid=runid, silent=True)
        
        WOA_input = WOAdata(self.runname,self.resultpath,self.mesh,self.ncfileSAL,'s_an', get_overview=False)
        sal_int = WOA_input.woa_int  
        
        WOA_input = WOAdata(self.runname,self.resultpath,self.mesh,self.ncfileTEMP,'t_an', get_overview=False)
        temp_int = WOA_input.woa_int  

        SALwoa = np.copy(sal_int)
        SALwoa[SALfesom == 0] = 0
        
        TEMPwoa = np.copy(temp_int)
        TEMPwoa[TEMPfesom == 0] = 0
        
        # Load and derive profiles

        nod_area = np.ma.masked_equal(meshdiag.nod_area.values, 0)
        mask = pf.get_mask(mesh, "Global Ocean")

        TEMPfesom_by_area = ((np.ma.masked_equal(TEMPfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        TEMPwoa_by_area = ((np.ma.masked_equal(TEMPwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        TEMPfesom_weighted_Global = TEMPfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        TEMPwoa_weighted_Global = TEMPwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

        SALfesom_by_area = ((np.ma.masked_equal(SALfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
        SALwoa_by_area = ((np.ma.masked_equal(SALwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

        SALfesom_weighted_Global = SALfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        SALwoa_weighted_Global = SALwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
        
        if regional:
            mask = pf.get_mask(mesh, "Atlantic_Basin")

            TEMPfesom_by_area = ((np.ma.masked_equal(TEMPfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            TEMPwoa_by_area = ((np.ma.masked_equal(TEMPwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            TEMPfesom_weighted_Atlantic = TEMPfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            TEMPwoa_weighted_Atlantic = TEMPwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            SALfesom_by_area = ((np.ma.masked_equal(SALfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            SALwoa_by_area = ((np.ma.masked_equal(SALwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            SALfesom_weighted_Atlantic = SALfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            SALwoa_weighted_Atlantic = SALwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            mask = pf.get_mask(mesh, "Pacific_Basin")

            TEMPfesom_by_area = ((np.ma.masked_equal(TEMPfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            TEMPwoa_by_area = ((np.ma.masked_equal(TEMPwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            TEMPfesom_weighted_Pacific = TEMPfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            TEMPwoa_weighted_Pacific = TEMPwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            SALfesom_by_area = ((np.ma.masked_equal(TEMPfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            SALwoa_by_area = ((np.ma.masked_equal(TEMPwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            SALfesom_weighted_Pacific = SALfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            SALwoa_weighted_Pacific = SALwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            mask = pf.get_mask(mesh, "Indian_Basin")

            TEMPfesom_by_area = ((np.ma.masked_equal(TEMPfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            TEMPwoa_by_area = ((np.ma.masked_equal(TEMPwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            TEMPfesom_weighted_Indian = TEMPfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            TEMPwoa_weighted_Indian = TEMPwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            SALfesom_by_area = ((np.ma.masked_equal(SALfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            SALwoa_by_area = ((np.ma.masked_equal(SALwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            SALfesom_weighted_Indian = SALfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            SALwoa_weighted_Indian = SALwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            mask = pf.get_mask(mesh, "Arctic_Basin")

            TEMPfesom_by_area = ((np.ma.masked_equal(TEMPfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            TEMPwoa_by_area = ((np.ma.masked_equal(TEMPwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            TEMPfesom_weighted_Arctic = TEMPfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            TEMPwoa_weighted_Arctic = TEMPwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            SALfesom_by_area = ((np.ma.masked_equal(SALfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            SALwoa_by_area = ((np.ma.masked_equal(SALwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            SALfesom_weighted_Arctic = SALfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            SALwoa_weighted_Arctic = SALwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            mask = pf.get_mask(mesh, "Southern_Ocean_Basin")

            TEMPfesom_by_area = ((np.ma.masked_equal(TEMPfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            TEMPwoa_by_area = ((np.ma.masked_equal(TEMPwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            TEMPfesom_weighted_Southern = TEMPfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            TEMPwoa_weighted_Southern = TEMPwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)

            SALfesom_by_area = ((np.ma.masked_equal(SALfesom[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))
            SALwoa_by_area = ((np.ma.masked_equal(SALwoa[mask,:],0) * nod_area[:-1,:].T[mask]).mean(axis=0))

            SALfesom_weighted_Southern = SALfesom_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            SALwoa_weighted_Southern = SALwoa_by_area/nod_area[:-1,:].T[mask].mean(axis=0)
            
            # plotting
            
            fig, axs = plt.subplots(2,6, figsize=(14, 7), facecolor='w', edgecolor='k', constrained_layout=True, sharey=True)

            axs[0,0].plot(TEMPfesom_weighted_Global, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'k', lw=3)
            axs[0,0].plot(TEMPwoa_weighted_Global, mesh.zlev[:-1]/1000,label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[0,0].set_ylabel('Depth [km]',fontsize=14)
            axs[0,0].set_xlabel(unittemp,fontsize=14)
            axs[0,0].set_title('Global Ocean',size=16, weight='bold')
            axs[0,0].tick_params(labelsize=14)
            axs[0,0].grid()
            axs[0,0].legend(loc='best', borderaxespad=0., fontsize=14)
            
            axs[1,0].plot(SALfesom_weighted_Global, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'k', lw=3)
            axs[1,0].plot(SALwoa_weighted_Global, mesh.zlev[:-1]/1000,label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[1,0].set_ylabel('Depth [km]',fontsize=14)
            axs[1,0].set_xlabel(unitsal,fontsize=14)
            axs[1,0].tick_params(labelsize=14)
            axs[1,0].grid()
            
            axs[0,1].plot(TEMPfesom_weighted_Pacific, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C0', lw=3)
            axs[0,1].plot(TEMPwoa_weighted_Pacific, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C0', lw=3, linestyle = '--')
            axs[0,1].set_xlabel(unittemp,fontsize=14)
            axs[0,1].set_title('Pacific Ocean',size=16, weight='bold')
            axs[0,1].tick_params(labelsize=14)
            axs[0,1].grid()
            
            axs[1,1].plot(SALfesom_weighted_Pacific, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C0', lw=3)
            axs[1,1].plot(SALwoa_weighted_Pacific, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C0', lw=3, linestyle = '--')
            axs[1,1].set_xlabel(unitsal,fontsize=14)   
            axs[1,1].tick_params(labelsize=14)
            axs[1,1].grid()
            
            axs[0,2].plot(TEMPfesom_weighted_Atlantic, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C1', lw=3)
            axs[0,2].plot(TEMPwoa_weighted_Atlantic, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C1', lw=3, linestyle = '--')
            axs[0,2].set_xlabel(unittemp,fontsize=14)
            axs[0,2].set_title('Atlantic Ocean',size=16, weight='bold')
            axs[0,2].tick_params(labelsize=14)
            axs[0,2].grid()
            
            axs[1,2].plot(SALfesom_weighted_Atlantic, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C1', lw=3)
            axs[1,2].plot(SALwoa_weighted_Atlantic, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C1', lw=3, linestyle = '--')
            axs[1,2].set_xlabel(unitsal,fontsize=14)
            axs[1,2].tick_params(labelsize=14)
            axs[1,2].grid()
            
            axs[0,3].plot(TEMPfesom_weighted_Arctic, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C2', lw=3)
            axs[0,3].plot(TEMPwoa_weighted_Arctic, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C2', lw=3, linestyle = '--')
            axs[0,3].set_xlabel(unittemp,fontsize=14)
            axs[0,3].set_title('Arctic Ocean',size=16, weight='bold')
            axs[0,3].tick_params(labelsize=14)
            axs[0,3].grid()
            
            axs[1,3].plot(SALfesom_weighted_Arctic, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C2', lw=3)
            axs[1,3].plot(SALwoa_weighted_Arctic, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C2', lw=3, linestyle = '--')
            axs[1,3].set_xlabel(unitsal,fontsize=14)
            axs[1,3].tick_params(labelsize=14)
            axs[1,3].grid()
            
            axs[0,4].plot(TEMPfesom_weighted_Southern, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C3', lw=3)
            axs[0,4].plot(TEMPwoa_weighted_Southern, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C3', lw=3, linestyle = '--')
            axs[0,4].set_xlabel(unittemp,fontsize=14)
            axs[0,4].set_title('Southern Ocean',size=16, weight='bold')
            axs[0,4].tick_params(labelsize=14)
            axs[0,4].grid()
            
            axs[1,4].plot(SALfesom_weighted_Southern, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C3', lw=3)
            axs[1,4].plot(SALwoa_weighted_Southern, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C3', lw=3, linestyle = '--')
            axs[1,4].set_xlabel(unitsal,fontsize=14)
            axs[1,4].tick_params(labelsize=14)
            axs[1,4].grid()
            
            axs[0,5].plot(TEMPfesom_weighted_Indian, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C4', lw=3)
            axs[0,5].plot(TEMPwoa_weighted_Indian, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C4', lw=3, linestyle = '--')
            axs[0,5].set_xlabel(unittemp,fontsize=14)
            axs[0,5].set_title('Indian Ocean',size=16, weight='bold')
            axs[0,5].tick_params(labelsize=14)
            axs[0,5].grid()
            
            axs[1,5].plot(SALfesom_weighted_Indian, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'C4', lw=3)
            axs[1,5].plot(SALwoa_weighted_Indian, mesh.zlev[:-1]/1000,label = 'WOA', color = 'C4', lw=3, linestyle = '--')
            axs[1,5].set_xlabel(unitsal,fontsize=14)
            axs[1,5].tick_params(labelsize=14)
            axs[1,5].grid()
            
            axs[0,0].set_ylim(-self.maxdepth,0)
            axs[1,0].set_ylim(-self.maxdepth,0)
            
        else:
            
            fig, axs = plt.subplots(1,2, figsize=(10, 5), facecolor='w', edgecolor='k', constrained_layout=True, sharey=True)

            axs[0].plot(TEMPfesom_weighted_Global, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'k', lw=3)
            axs[0].plot(TEMPwoa_weighted_Global, mesh.zlev[:-1]/1000,label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[0].set_ylabel('Depth [km]',fontsize=14)
            axs[0].set_xlabel(unittemp,fontsize=14)
            axs[0].set_title('Global Ocean',size=16, weight='bold')
            axs[0].tick_params(labelsize=14)
            axs[0].grid()
            
            axs[1].plot(SALfesom_weighted_Global, mesh.zlev[:-1]/1000,label = 'FESOM', color = 'k', lw=3)
            axs[1].plot(SALwoa_weighted_Global, mesh.zlev[:-1]/1000,label = 'WOA', color = 'k', lw=3, linestyle = '--')
            axs[1].set_ylabel('Depth [km]',fontsize=14)
            axs[1].set_xlabel(unitsal,fontsize=14)
            axs[1].tick_params(labelsize=14)
            axs[1].set_title('Global Ocean',size=16, weight='bold')
            axs[1].grid()
            axs[1].legend(bbox_to_anchor=(1.1, 0.5), loc='lower left', borderaxespad=0., fontsize=14)
            
            axs[0].set_ylim(-self.maxdepth,0)
            axs[1].set_ylim(-self.maxdepth,0)
        
        if(savefig):
                plt.savefig(self.savepath+self.runname+'_TS_profiles_'+str(self.fyear)+'to'+str(self.lyear)+'.png', dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_TS_profiles_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', bbox_inches='tight')
        plt.show(block=False)